In [ ]:
# check the current environment
using Pkg; Pkg.status()

## Circuit loading
We provide shallow circuits in the `data` folder, they are some popular quantum circuits for demonstrating quantum supremacy. These circuit are copied from [qfelx](https://github.com/s-mandra/qflex). To load the circuits to Yao.jl, please use the `YaoQASMReader` module provided in file `reader.jl`:

In [ ]:
using Yao

# circuit reader
include("reader.jl")
using .YaoCircuitReader: yaocircuit_from_file

In [ ]:
readdir(joinpath(@__DIR__, "data", "circuits"))

We load the Sycamore 8 layer circuit data to Julia with [Yao](https://github.com/QuantumBFS/Yao.jl) (幺), a high performance quantum simulator. Sycamore circuit is a famous circuit that used in the random circuit benchmarking experiment to showcase the quantum advantage.

**Reference**: Arute, et al, 2019. Quantum supremacy using a programmable superconducting processor. Nature 574, 505–510. https://doi.org/10.1038/s41586-019-1666-5

In [ ]:
# please replace "test.txt" with "bristlecone_70_1-12-1_0.txt", a circuit with 70 qubits, 12 layers
filename = joinpath(@__DIR__, "data", "circuits", "bristlecone_70_1-20-1_0.txt")

In [ ]:
# load a visualize
@info "Circuit: $(filename)"
c = yaocircuit_from_file(filename)
vizcircuit(c)

## Case 1: compute <0|c|0> with tensor networks

During the convertion, we also specify an optimizer to specify the contraction order.

In [ ]:
# all input and output qubits are set to |0>
initial_state = Dict(zip(1:nqubits(c), zeros(Int,nqubits(c))))
final_state = Dict(zip(1:nqubits(c), zeros(Int,nqubits(c))))
optimizer = TreeSA(ntrials=1)  # contraction order optimizer
net = yao2einsum(c; initial_state, final_state, optimizer)
@info "Contraction complexity: $(contraction_complexity(net))"

The space complexity is the number of elements in the largest itermediate tensor. It is a much smaller number compared with the full amplitude simulation that costs 2^(# of qubits) in storage. Learn more about contraction order optimizers: https://tensorbfs.github.io/OMEinsumContractionOrders.jl/dev/optimizers/

In [ ]:
fieldnames(typeof(net))

In [ ]:
length(net.tensors)

In [ ]:
psi0 = contract(net)

In [ ]:
# using OMEinsum.OMEinsumContractionOrders: viz_eins
# using LuxorGraphPlot
# viz_eins(net.code)

## Case 2: Add noise and compute <ψ|X₁X₂|ψ>, where |ψ> = c |0>

During the convertion, we also specify an optimizer to specify the contraction order.

In [ ]:
# ## Case 2: add noise and compute <ψ|X₁X₂|ψ>, where |ψ> = c |0>
function add_depolarizing_noise(c::AbstractBlock, depolarizing)
    Optimise.replace_block(c) do blk
        if blk isa PutBlock || blk isa ControlBlock
            rep = chain(blk)
            for loc in occupied_locs(blk)
                push!(rep, put(nqubits(blk), loc=>DepolarizingChannel(1, depolarizing)))
            end
            return rep
        else
            return blk
        end
    end
end
noisy_c = add_depolarizing_noise(c, 0.01)
vizcircuit(noisy_c)


In [ ]:
initial_state = Dict(zip(1:nqubits(noisy_c), zeros(Int,nqubits(noisy_c))))
observable = kron(nqubits(noisy_c), 1=>X, 2=>X)
noisy_net = yao2einsum(noisy_c; initial_state, observable, optimizer, mode=DensityMatrixMode())
@info "Contraction complexity: $(contraction_complexity(noisy_net))"

In [ ]:
# contract(noisy_net)
# TODO: add ref: xun gao

## References
- Ayral, T., Louvet, T., Zhou, Y., Lambert, C., Stoudenmire, E.M., Waintal, X., 2023. Density-Matrix Renormalization Group Algorithm for Simulating Quantum Circuits with a Finite Fidelity. PRX Quantum 4, 020304. https://doi.org/10.1103/PRXQuantum.4.020304
- B. Villalonga, et al., "A flexible high-performance simulator for verifying and benchmarking quantum circuits implemented on real hardware", NPJ Quantum Information 5, 86 (2019)